**Id:** 55031 **Nome:** RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI<br>
**Id:** 52662 **Nome:** CÁLCULO DIFERENCIAL E INTEGRAL I<br>
**Id:** 55022 **Nome:** FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I<br>

In [2]:
import numpy as np
import pandas as pd

In [3]:
#Função que insere 'vazio' no campo null. Sem isso os dataframes iram quebrar.
def TratarDfVaziosProf(df):
    if(df.empty):
        df = pd.DataFrame()
        df = df.append({'id_servidor':0, 'nome':'vazio', 'formacao':'vazio', 'tipo_jornada_trabalho':'vazio', 'vinculo':'vazio', 'classe_funcional':'vazio', 'lotacao': 'vazio'}, ignore_index=True)
        return df
    return df

In [4]:
pd_disc = pd.read_csv('disciplinas.csv')
pd_docentes = pd.read_csv('docentes.csv', encoding = "utf-8", sep=";", usecols=['id_servidor', 'nome', 'formacao', 'tipo_jornada_trabalho', 'vinculo', 'lotacao'])
all_turma_matriculados = pd.read_csv('all_turma_matriculados.csv', encoding = "utf-8", sep=",")

In [5]:
framesAux = []
for index_turma, row_turma in all_turma_matriculados.iterrows():
    pd_turmas = pd.read_csv(row_turma['csv_turma'], error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8", sep=";", usecols=['id_turma', 'codigo_turma', 'id_docente_interno', 'id_componente_curricular', 'local', 'ano', 'periodo', 'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'situacao_turma'])
    framesInter = []
    for index_pd_disc, row_pd_disc in pd_disc.iterrows():

        id_disc = row_pd_disc['id_disc']
        list_turmas = pd_turmas[pd_turmas['id_componente_curricular'] == id_disc] # filtra todas as turmas com o id da disciplina
        list_turmas_disc = list_turmas.assign(nome_disc=row_pd_disc['nome_disc']) # add uma coluna com o nome da disciplina

        listProf = []
        for index, row in list_turmas_disc.iterrows():
            prof = pd_docentes[pd_docentes['id_servidor'] == int(row["id_docente_interno"])]
            listProf.append(TratarDfVaziosProf(prof))

        df_profs = pd.concat(listProf)
        list_turmas_disc_prof = list_turmas_disc.assign(
            nome_prof = df_profs['nome'].tolist(),
            id_prof = df_profs['id_servidor'].tolist(),
            formacao_prof = df_profs['formacao'].tolist(),
            tipo_jornada_trabalho_prof = df_profs['tipo_jornada_trabalho'].tolist(),
            vinculo_prof = df_profs['vinculo'].tolist(),
            lotacao_prof = df_profs['lotacao'].tolist())
        framesInter.append(list_turmas_disc_prof)
    
    df_turma_professorAux = pd.concat(framesInter) # concatena todos os dataframes
    framesAux.append(df_turma_professorAux)
    
df_turma_professor = pd.concat(framesAux) 
df_turma_professor.to_csv('turma_professor.csv', encoding='UTF-8')

In [6]:
#url = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171.csv'
#pd_alunos_2017_1 = pd.read_csv(url, encoding = "utf-8", sep=";")
#pd_alunos_2017_1.head(3)

frames = []
for item in df_turma_professor:
    turma = item['id_turma']
    
    df_turma_professor[df_turma_professor['id_turma'] == ]
    df_alunos = 0
    for index, row in all_turma_matriculados.iterrows():
        pd_alunos = pd.read_csv(row['csv_matriculados'], encoding = "utf-8", sep=";")  
        if(not pd_alunos[pd_alunos['id_turma'] == item].empty):
            df_alunos = pd_alunos[pd_alunos['id_turma'] == item]
            break
    frames.append(df_alunos)
    
df_alunos = pd.concat(frames) # concatena todos os dataframes
df_alunos.to_csv('alunos.csv', encoding='UTF-8')
    

SyntaxError: invalid syntax (<ipython-input-6-b5f1546831c2>, line 9)

In [ ]:
frames = []
for item in df_turma_professor['id_turma'].unique():
    df_alunos = pd_alunos_2017_1[pd_alunos_2017_1['id_turma'] == item]
    frames.append(df_alunos)
df_alunos = pd.concat(frames) # concatena todos os dataframes
df_alunos.to_csv('alunos_2017_1.csv', encoding='UTF-8')
#df_alunos.head(5)

In [ ]:
listMean = []
listMin = []
list1q = []
list2q = []
list3q = []
listmax = []

for turma in df_turma_professor_2017_1['id_turma']:
    df_turma_alunos_3notas = df_alunos[df_alunos['id_turma'] == turma] #nesse df tá com as 3 notas do aluno
    
    if(df_turma_alunos_3notas['discente'].size > 0):
        
        df_turma_alunos_final = df_turma_alunos_3notas.drop_duplicates(subset=['discente']) #retiro 2 linhas do mesmo aluno.
        media_final_summary = df_turma_alunos_final.describe()['media_final']
        
        listMean.append(media_final_summary['mean'])
        listMin.append(media_final_summary['min'])
        list1q.append(media_final_summary['25%'])
        list2q.append(media_final_summary['50%'])
        list3q.append(media_final_summary['75%'])
        listmax.append(media_final_summary['max'])

    else: #ocorre quando não encontra uma turma
        listMean.append(0)
        listMin.append(0)
        list1q.append(0)
        list2q.append(0)
        list3q.append(0)
        listmax.append(0)


df_turma_professor_2017_1 = df_turma_professor_2017_1.assign(
    disc_mean = listMean,
    disc_min = listMin,
    disc_q1 = list1q,
    disc_q2 = list2q,
    disc_q3 = list3q,
    disc_max = listmax
    )
df_turma_professor_2017_1.to_csv('turma_professor_2017_1.csv', encoding='UTF-8')
#df_turma_professor_2017_1